In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
%%time 
train_data = pd.read_csv("data/ohe_train_recipes_v2.csv",index_col="id")
test_data = pd.read_csv("data/ohe_test_recipes_v2.csv",index_col="id")

In [3]:
train_data.head(2)

,1% buttermilk,1% chocolate milk,1% cottage cheese,1% milk,"2 1/2 to 3 lb. chicken, cut into serving pieces",2% cottage cheese,2% low fat cheddar chees,2% lowfat greek yogurt,2% milk mozzarella cheese,2% reduced-fat milk,...,yuzu,yuzu juice,za'atar,zest,zesty italian dressing,zinfandel,ziti,zucchini,zucchini blossoms,cuisine
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,spanish
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,mexican


In [4]:
X=train_data.drop(['cuisine'],axis=1)
response=train_data['cuisine']

In [20]:
X_train,X_validate,y_train,y_validate=train_test_split(X,response,test_size=0.5,random_state=22)

In [21]:
X_train.shape[0], X_validate.shape[0], X_validate.shape[0]/(X_validate.shape[0]+X_train.shape[0])

(19887, 19887, 0.5)

In [32]:
%%time
lr=LogisticRegression(C=0.05)
lr.fit(X_train,y_train)

CPU times: user 3min 7s, sys: 1.95 s, total: 3min 9s
Wall time: 26.8 s


LogisticRegression(C=0.05)

In [33]:
lr.score(X_validate,y_validate), lr.score(X_train,y_train)

(0.7096595766078343, 0.7479760647659275)

In [8]:
from sklearn.preprocessing import StandardScaler

In [10]:
scaler = StandardScaler()
X_train_trans = scaler.fit_transform(X_train)

In [11]:
X_validate_trans = scaler.transform(X_validate)

In [12]:
%%time
lr=LogisticRegression()
lr.fit(X_train_trans,y_train)

CPU times: user 6min 59s, sys: 4.02 s, total: 7min 3s
Wall time: 59 s


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [13]:
lr.score(X_validate,y_validate), lr.score(X_train,_train)

0.6918744971842317

In [34]:
from sklearn.model_selection import GridSearchCV

In [35]:
parameters = {'C':[0.05,0.1,0.5,1,2]}
lr=LogisticRegression()

In [36]:
clf = GridSearchCV(lr, parameters)

In [ ]:
clf.fit(X_train, y_train)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l